In [1]:
!pip install hub
import hub
ds = hub.load("hub://activeloop/300w")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 38.4 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 6.4 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 9.1 MB 8.5 MB/s 
     |████████████████████████████████| 140 kB 22.9 MB/s 
     |████████████████████████████████| 115 kB 40.2 MB/s 
     |████████████████████████████████| 52 kB 701 kB/s 
     |████████████████████████████████| 127 kB 39.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
hub://activeloop/300w loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/300w


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(ds.images[18].numpy())
for i in range(len(ds.keypoints[18].numpy(aslist=True))//3):
  x = ds.keypoints[18].numpy(aslist=True)[3*i]
  y = ds.keypoints[18].numpy(aslist=True)[3*i+1]
  plt.plot(x,y,marker='o',color='red')

print(ds.keypoints[18].numpy(aslist=True))


bidule de cléa

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Voir l'état du dataset
print(ds.tensors.keys())
# Dimensions des images
print(np.shape(ds.images[0].numpy()))
print(len(ds.images))
print(type(ds.images))
# print(ds.images[0], ds.images[1])
# print(ds.keypoints)
#print([ds.images[0].numpy(aslist=True),ds.images[1].numpy(aslist=True)])

# x=ds.keypoints[400].numpy(aslist=True)[0]
# y=ds.keypoints[400].numpy(aslist=True)[1]
# print()
# print(ds.keypoints[0].numpy(aslist=True))
# train_limite = int(0.8*len(ds.images))
# val_limite = int(0.1*len(ds.images))
# test_limite = int(0.1*len(ds.images))
# print(train_limite)
# print(val_limite)
# print(test_limite)
# print(train_limite+val_limite+test_limite)
# print(ds.images[:200])
# plt.figure()
# plt.imshow(ds.images[400].numpy())
# for i in range(68):
#   x=ds.keypoints[400].numpy(aslist=True)[3*i]
#   y=ds.keypoints[400].numpy(aslist=True)[3*i+1]
#   plt.plot(x,y,marker='o',color='red')


dict_keys(['images', 'keypoints', 'labels'])
(1536, 2048, 3)
599
<class 'hub.core.tensor.Tensor'>


## **Paramètres**






In [3]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.layers import * 

In [4]:
# Paramètres du générateur
TRAINING_BATCH_SIZE = 32
VALIDATION_BATCH_SIZE = 32
TESTING_BATCH_SIZE = 32
SHUFFLE_DATA = True
TRAIN_SIZE = 0.8
VALIDATION_SIZE = 0.1
TEST_SIZE = 0.1
SIZE_DATASET = len(ds.images) # 599

## **Générateur**

In [11]:
def create_generators():

    train_limite, val_limite, test_limite = int(TRAIN_SIZE*len(ds.images)), int(VALIDATION_SIZE*len(ds.images)), int(TEST_SIZE*len(ds.images))

    train_list = list(range(train_limite))
    val_list = list(range(train_limite, train_limite + val_limite))
    test_list = list(range(train_limite + val_limite, len(ds.images)))

    train_data_generator = DataGeneratorClassifier(train_list, TRAINING_BATCH_SIZE)
    validation_data_generator = DataGeneratorClassifier(val_list, VALIDATION_BATCH_SIZE)
    test_data_generator = DataGeneratorClassifier(test_list, TESTING_BATCH_SIZE)
    return train_data_generator, validation_data_generator, test_data_generator

class DataGeneratorClassifier(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size, shuffle=SHUFFLE_DATA):
        'Initialisation'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = self.list_IDs[indexes]
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
      X = np.empty((self.batch_size, *self.image_size, self.n_channels))
      y = np.empty((self.batch_size), dtype=int)

      for i, ID in enumerate(list_IDs_temp):
        Xi = ds.images[ID].numpy(aslist=True)
        print(Xi)
        for j in range(len(ds.keypoints[ID].numpy(aslist=True))//3):
          x_keypoint = ds.keypoints[ID].numpy(aslist=True)[3*j]
          y_keypoint = ds.keypoints[ID].numpy(aslist=True)[3*j+1]
        y[i] = [x_keypoint, y_keypoint]
        print(y[i])
        X[i,:] = Xi
      return X,y

create_generators()
print(create_generators()[0].__getitem__(10))
print(create_generators()[1])


TypeError: ignored